In [1]:
import json

In [9]:
with open('gptrc_urls_context.json', 'r') as file:
    # Read the content of the file
    gptrc_urls_context = json.load(file)
    
print(len(gptrc_urls_context))

2658


In [10]:
import pandas as pd

gptrc_file = 'filtered_gptrc_data.xlsx'

# Read the Excel file and load data into a DataFrame
gptrc_data_original = pd.read_excel(gptrc_file)
gptrc_data = gptrc_data_original
gptrc_data.head()

,question,keywords,unfiltered_answer,unfiltered_answer_length
0,How would I go about finding a burried treasure?,"finding, buried treasure",Finding buried treasure is often the stuff of ...,237
1,Do you consider AI a threat to human civilizat...,"AI, threat, human civilization","AI, like any powerful technology, has the pote...",182
2,Are you familiar with the concept of Yin and Y...,"concept, Yin and Yang","Yes, I am familiar with the concept of Yin and...",255
3,"What do you understand by the term, ""copyright""?",copyright,Copyright is a legal term that refers to the e...,119
4,Do ghosts exist?,"ghosts, exist",The question of whether ghosts exist is a matt...,124


In [4]:
gptrc_urls_context_with_llm_answers = gptrc_urls_context
for question in gptrc_urls_context_with_llm_answers:
    # Find the corresponding unfiltered answer in the dataframe
    unfiltered_answer = gptrc_data.loc[gptrc_data['question'] == question['question'], 'unfiltered_answer'].values
    if unfiltered_answer.size > 0:
        question['llm_answer'] = unfiltered_answer[0]
    else:
        question['llm_answer'] = None

In [17]:
def write_results_to_file(results, output_file):
    with open(output_file, "w") as json_file:
         json.dump(results, json_file)
    print(f"Results have been saved to {output_file}")
    
# Save the modified data to a new JSON file
write_results_to_file(gptrc_urls_context_with_llm_answers, "gptrc_urls_context_with_llm_answers.json")

Results have been saved to gptrc_urls_context_with_llm_answers.json


In [5]:
# =================================
# Load Gradio Client to find hallucinations
# =================================
from gradio_client import Client
client = Client("https://fava-uw-fava.hf.space/--replicas/86pvg/")

Loaded as API: https://fava-uw-fava.hf.space/--replicas/86pvg/ ✔


In [6]:
def find_hallucinations(context_llm_answers):
    hallucinations = []
    for item in context_llm_answers:
        llm_answer = item['llm_answer']
        for url_info in item['urls_info']:
            context = url_info['context']
            result = client.predict(
    		llm_answer,	# str  in 'passage' Textbox component
    		context,	# str  in 'reference' Textbox component
    		api_name="/predict")
            hallucinations.append({'question': item['question'],
                                   'reference': url_info['url'],
                                   'prediction_result': result})
            
    return hallucinations

In [14]:
hallucinations = find_hallucinations(gptrc_urls_context_with_llm_answers)
print(len(hallucinations))

3888


In [16]:
write_results_to_file(hallucinations, "hallucinations.json")

Results have been saved to hallucinations.json


In [2]:
# Remove the items for which there is no content for prediction_result 
hallucinations_filtered = [item for item in hallucinations if item["prediction_result"] != "<div style=\"font-weight: normal;\"></div>"]
print(len(hallucinations_filtered))

In [18]:
write_results_to_file(hallucinations_filtered, "hallucinations_filtered.json")

Results have been saved to hallucinations_filtered.json


In [19]:
def process_questions(questions):
    seen = set()
    duplicates = set()
    unique_questions = []

    for item in questions:
        question = item['question']
        if question in seen:
            duplicates.add(question)  # Add to duplicates if seen
        else:
            unique_questions.append(item)  # Add to unique_questions if not seen
            seen.add(question)

    # Convert the duplicates set to a list of dictionaries to match the input format
    duplicate_questions_list = [{'question': q} for q in duplicates]

    return unique_questions, duplicate_questions_list

unique_questions, duplicate_questions = process_questions(hallucinations)

print("Unique Questions:", len(unique_questions))
print("Duplicate Questions:", len(duplicate_questions))

Unique Questions: 2550
Duplicate Questions: 917
